In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import os
path="/content/drive/My Drive/ComputerSelfStudies/DL4CV/DL_CV2/Chapter_02/"
os.chdir(path)
from utilities.preprocessing import ImageToArrayPreprocessor
from utilities.preprocessing import AspectAwarePreprocessor
from utilities.datasets import SimpleDatasetLoader
from utilities.nn.cnn import MiniVGGNet
from utilities.nn.cnn import LeNet
from utilities.nn.cnn import ShallowNet
import cv2
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD
from keras.models import load_model
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import os
from google.colab.patches import cv2_imshow

Using TensorFlow backend.


In [4]:
!ls

 Augmented_Flower17.ipynb   'Learning Rate: 0.025.png'
 Combined_image.png	    'Learning Rate: 0.05.png'
 flowers17.zip		    'Learning Rate: 0.1.png'
'Learning Rate: 0.001.png'  'Learning Rate: 0.5.png'
'Learning Rate: 0.003.png'   models
'Learning Rate: 0.007.png'   PredictionAnalysis.ipynb
'Learning Rate: 0.013.png'   utilities


In [0]:
!unzip -q flowers17.zip

In [0]:
imagePaths = list(paths.list_images(path+"flowers17"))
classNames = [pt.split(os.path.sep)[-2] for pt in imagePaths]
classNames = [str(x) for x in np.unique(classNames)]

In [7]:
for names in classNames:
  print(names)

blue1
blue2
mix1
purple1
purple2
white1
white2
white3
white4
yellow1
yellow2
yellow3
yellow4
yellow5
yellow6
yellow7
yellow8


In [0]:
aap = AspectAwarePreprocessor(64, 64)
iap = ImageToArrayPreprocessor()

In [9]:
# load the dataset from disk then scale the raw pixel intensities
# to the range [0, 1]
sdl = SimpleDatasetLoader(preprocessors=[aap, iap])
(data, labels) = sdl.load(imagePaths, verbose=500)
data = data.astype("float") / 255.0
# partition the data into training and testing splits using 75% of
# the data for training and the remaining 25% for testing
(trainX, testX, trainY, testY) = train_test_split(data, labels,test_size=0.25, random_state=42)
# convert the labels from integers to vectors
trainY = LabelBinarizer().fit_transform(trainY)
testY = LabelBinarizer().fit_transform(testY)

[INFO]: Processed 500/1360
[INFO]: Processed 1000/1360


In [0]:
# construct the image generator for data augmentation
aug = ImageDataGenerator(rotation_range=30, width_shift_range=0.1,height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,horizontal_flip=True, fill_mode="nearest")

In [0]:
List_of_models=[MiniVGGNet,ShallowNet,LeNet]
List_of_LRs=[0.001,0.003,0.007,0.013,0.025,0.05,0.1,0.5]
Names=["MiniVGG","ShallowNet","LeNet"]
Main_List=[]
Main_Reports=[]

In [0]:
for i in range(len(List_of_LRs)):
  opt = SGD(lr=List_of_LRs[i])
  List=[]
  Reports=[]
  j=0
  for Model in List_of_models:
    model = Model.build(width=64, height=64, depth=3,classes=len(classNames))
    model.compile(loss="categorical_crossentropy", optimizer=opt,metrics=["accuracy"])
    List.append(model.fit_generator(aug.flow(trainX, trainY, batch_size=32),validation_data=(testX, testY), steps_per_epoch=len(trainX) // 32,epochs=100, verbose=0))
    predictions = model.predict(testX, batch_size=32)
    Reports.append(classification_report(testY.argmax(axis=1),predictions.argmax(axis=1), target_names=classNames,output_dict=1))
    model.save("models/"+str(List_of_LRs[i])+"_"+Names[j]+".hdf5")
    j+=1
  Main_List.append(List)
  Main_Reports.append(Reports)

In [0]:
plt.style.use("ggplot")
for i in range(len(List_of_LRs)):
  fig, ax = plt.subplots(1, len(Names), figsize=(21, 7))
  Label="Learning Rate: "+str(List_of_LRs[i])
  fig.suptitle(Label)
  for j in range(len(Names)):
    ax[j].plot(np.arange(0, 100), Main_List[i][j].history["loss"],color='green', label="train_loss")
    ax[j].plot(np.arange(0, 100), Main_List[i][j].history["val_loss"],color='blue', label="val_loss")
    ax[j].plot(np.arange(0, 100), Main_List[i][j].history["acc"], color='orange',label="train_acc")
    ax[j].plot(np.arange(0, 100), Main_List[i][j].history["val_acc"], color='purple',label="val_acc")
    ax[j].set_title(Names[j]+": Acc "+str(round(Main_Reports[i][j]["accuracy"],2)) + " , Prec "+ str(round(Main_Reports[i][j]["weighted avg"]["precision"],2))+" , ReCa "+str(round(Main_Reports[i][j]["weighted avg"]["recall"],2) ) )
    ax[j].set_xlabel(xlabel='Epoch #')
    ax[j].set_ylabel(ylabel='Loss/Accuracy')
    ax[j].legend(loc='upper right')
    ax[j].axhline(1, color='black', lw=2,ls="--")
  x=plt.savefig(Label+".png")

In [0]:
List_of_images=[]
for stuff in os.listdir():
  if(stuff[-3:-1]=="pn"):
    List_of_images.append(cv2.imread(stuff))
vis = np.concatenate(List_of_images, axis=0)
cv2.imwrite('Combined_image.png',vis)

True

In [0]:
path="/content/drive/My Drive/ComputerSelfStudies/DL4CV/DL_CV2/Chapter_02/models"
os.chdir(path)

In [0]:
Accuracy={}
Recall={}
Precision={}

for name in Names:
  Accuracy[name]=[]
  Recall[name]=[]
  Precision[name]=[]

In [0]:
for path in os.listdir():
  [LR,Type]=path.split("_")
  Type=Type.split(".")[0]
  print (LR+" "+Type)
  model = load_model(path)
  predictions = model.predict(testX, batch_size=32)
  report = classification_report(testY.argmax(axis=1),predictions.argmax(axis=1), target_names=classNames,output_dict=1)
  Accuracy[Type].append(round(report["accuracy"],2))
  Precision[Type].append(round(report["weighted avg"]["precision"],2))
  Recall[Type].append(round(report["weighted avg"]["recall"],2))

In [43]:
List_of_LRs_string=[]
for LR in List_of_LRs:
  List_of_LRs_string.append(str(LR))

['0.001', '0.003', '0.007', '0.013', '0.025', '0.05', '0.1', '0.5']


In [0]:
path="/content/drive/My Drive/ComputerSelfStudies/DL4CV/DL_CV2/Chapter_02"
os.chdir(path)

Colors={"MiniVGG":"red","ShallowNet":"blue","LeNet":"purple"}

plt.style.use("ggplot")
fig, [Acc,Prec,Rec] = plt.subplots(1, len(Names), figsize=(21, 7))
for pic in [Acc,Prec,Rec]:
  pic.set_xscale('log')
fig.suptitle("Final Statistics")
for model_name in Names:
  Acc.plot(List_of_LRs, Accuracy[model_name], color=Colors[model_name],label=model_name)
  Rec.plot(List_of_LRs, Recall[model_name], color=Colors[model_name],label=model_name)
  Prec.plot(List_of_LRs, Precision[model_name], color=Colors[model_name],label=model_name)

  Acc.set_title("Accuracy")
  Acc.set_xlabel(xlabel='Learning Rate')
  Acc.set_ylabel(ylabel='Accuracy')
  Acc.legend(loc='bottom left')

  Prec.set_title("Precision")
  Prec.set_xlabel(xlabel='Learning Rate')
  Prec.set_ylabel(ylabel='Precision')
  Prec.legend(loc='bottom left')

  Rec.set_title("Recall")
  Rec.set_xlabel(xlabel='Learning Rate')
  Rec.set_ylabel(ylabel='Recall')
  Rec.legend(loc='bottom left')
plt.savefig("FinalStatistics.png")

In [75]:
!ls

 Augmented_Flower17.ipynb   'Learning Rate: 0.013.png'
 Combined_image.png	    'Learning Rate: 0.025.png'
 FinalStatistics.png	    'Learning Rate: 0.05.png'
 flowers17.zip		    'Learning Rate: 0.1.png'
'Learning Rate: 0.001.png'  'Learning Rate: 0.5.png'
'Learning Rate: 0.003.png'   models
'Learning Rate: 0.007.png'   utilities
